In [ ]:
import os
import chess as python_chess
from datetime import datetime
from dotenv import load_dotenv

from chess_game import ChessGame
from model.gaming_model import GamingRLModel
from model.local_model_saver import LocalModelSaver
from model.cloud_model_saver import CloudModelSaver
from chess_trainer import ChessTrainer
from engine import ChessEngine
from backblaze_gateway import BackblazeGateway

load_dotenv()

timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
filename = f"gaming_model_{timestamp}.keras"

application_key_id = os.getenv("B2_APPLICATION_KEY_ID")
application_key = os.getenv("B2_APPLICATION_KEY")
bucket_name = os.getenv("B2_BUCKET_NAME")

if not all([application_key_id, application_key, bucket_name]):
    raise ValueError("Missing Backblaze credentials in .env file")
gateway = BackblazeGateway(application_key_id, application_key, bucket_name)

model_saver = CloudModelSaver(filename, gateway)
# model_saver = LocalModelSaver(f"../{filename}")

model = GamingRLModel(model_saver, "../gaming_model.keras")
chess = ChessGame()
engine = ChessEngine(model)
trainer = ChessTrainer(engine)

board = chess.get_initial_state()
board.push(python_chess.Move.from_uci("e2e4"))


In [ ]:
move,_,_ = engine.best_move(board, 100, 0)
print(move)
board.push(move)

In [ ]:
board = chess.get_initial_state()

# results = trainer.play_game(board, 100)
results = trainer.generate_games("../games_data", 720, 25)
print(results)

In [ ]:
trainer.save_games(results, "../games_data")

In [ ]:
results = trainer.load_games("../games_data")

In [ ]:
flat_results = [step for game in results for step in game]

model.train(flat_results, None, 1)